In [ ]:
import numpy as np
import pandas as pd
import pickle


from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
#from modAL.models import Committee
#from modAL.disagreement import vote_entropy_sampling

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

import recordlinkage as rl

from active_learn import*
from thresholding import*

from IPython import display
from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
from importlib import reload
#reload(spam)

In [ ]:
#df_uns = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_uns_label_img_1.csv')
df_uns_red =pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_uns_label_img_red_hot.csv')

df_full = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\merged_clean_df.csv')
df_full.drop('Unnamed: 0',axis=1,inplace=True)

#Manually annotated data
test_1 = rl.read_annotation_file(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\result_bnb_trip.json')
test_2 = rl.read_annotation_file(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\result_bnb_vrb.json')

#Baseline with unsupervised
df_full_base = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_uns_label_base_train.csv')
df_full_base['label'] = df_full_base['label'].map(to_bin)

# Updated training df
#df_ams_train = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_ams_train_1.csv')

In [ ]:
# Divide test and training set for baseline 
df_base_train = df_full_base.loc[:1761] 
df_base_test = df_full_base.loc[1762:]

In [ ]:
#Baseline models data
not_train = ['source_id','target_id','ids', 'agg_score', 'weights','uns_label','label']
X_train = df_base_train.drop(not_train, axis=1).values
y_train = df_base_train['label'].values
X_test = df_base_test.drop(not_train, axis=1).values
y_test = df_base_test['label'].values

In [ ]:
#Decision Tress serach for hyperparameters
params = {'max_leaf_nodes': list(range(2, 100)),'max_depth' : [None, 3, 5, 10, 15], 
        'min_samples_leaf' : [3, 5, 10], 'criterion' : ['gini', 'entropy']}

grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)
#Best parameters
print(grid_search_cv.best_params_)
#Get the perd metrics of the model 
y_pred = grid_search_cv.predict(X_test)
precision_recall_fscore=precision_recall_fscore_support(y_test,y_pred,average='binary',zero_division=0)
print('Precision:',precision_recall_fscore[0],
      'Recall:',precision_recall_fscore[1],
      'Fscore:',precision_recall_fscore[2])

In [ ]:
# Logistic Reg search for hyperparameters
params = {'penalty' :['l1', 'l2'],'fit_intercept' : [True, False],
        'solver' :['liblinear', 'saga'],'max_iter' : [50, 100, 150]}

grid_search_cv = GridSearchCV(LogisticRegression(random_state=42), params, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)
#Best parameters
print(grid_search_cv.best_params_)
#Get the perd metrics of the model 
y_pred = grid_search_cv.predict(X_test)
precision_recall_fscore=precision_recall_fscore_support(y_test,y_pred,average='binary',zero_division=0)
print('Precision:',precision_recall_fscore[0],
      'Recall:',precision_recall_fscore[1],
      'Fscore:',precision_recall_fscore[2])

In [ ]:
# SVM search for hyperparameters
params = {'kernel':['linear', 'rbf'], 'gamma':[0.1, 1, 5, 10] ,'C' : [0.5, 1, 5, 10]}

grid_search_cv = GridSearchCV(SVC(random_state=42), params, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)
#Best parameters
print(grid_search_cv.best_params_)
#Get the perd metrics of the model 
y_pred = grid_search_cv.predict(X_test)
precision_recall_fscore=precision_recall_fscore_support(y_test,y_pred,average='binary',zero_division=0)
print('Precision:',precision_recall_fscore[0],
      'Recall:',precision_recall_fscore[1],
      'Fscore:',precision_recall_fscore[2])


In [ ]:
# XGBoost search for hyperparameters
params = {'n_estimators':[100, 150, 200],'learning_rate':[0.05, 0.1, 0.15],'max_depth':[3, 5, 7]}

grid_search_cv = GridSearchCV(xgb.XGBClassifier(random_state=42), params, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)
#Best parameters
print(grid_search_cv.best_params_)
#Get the perd metrics of the model 
y_pred = grid_search_cv.predict(X_test)
precision_recall_fscore=precision_recall_fscore_support(y_test,y_pred,average='binary',zero_division=0)
print('Precision:',precision_recall_fscore[0],
      'Recall:',precision_recall_fscore[1],
      'Fscore:',precision_recall_fscore[2])


In [ ]:
# Random Forest search for hyperparameters
params = {'n_estimators':[10, 50, 100] ,'max_depth':[None, 3, 5, 10, 15],'min_samples_leaf':[3, 5, 10]}

grid_search_cv = GridSearchCV(RandomForestClassifier(random_state=42), params, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)
#Best parameters
print(grid_search_cv.best_params_)
#Get the perd metrics of the model 
y_pred = grid_search_cv.predict(X_test)
precision_recall_fscore=precision_recall_fscore_support(y_test,y_pred,average='binary',zero_division=0)
print('Precision:',precision_recall_fscore[0],
      'Recall:',precision_recall_fscore[1],
      'Fscore:',precision_recall_fscore[2])